# Covid19 Analysis for Nordic Countries

In [32]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

from matplotlib.pyplot import figure
figure(num=None, figsize=(8, 6), dpi=80, facecolor='w', edgecolor='k')

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

In [33]:
df = pd.read_json('https://pomber.github.io/covid19/timeseries.json')[['Sweden','Norway','Denmark','Finland']]

In [34]:
n_rows = df.shape[0]
df['Sweden'][0]

{'date': '2020-1-22', 'confirmed': 0, 'deaths': 0, 'recovered': 0}

In [35]:
# extract dates
dates = []
for i in range(n_rows):
    dates.append(df['Sweden'][i]['date'])

In [36]:
df['Date'] = dates
df['Date'] = pd.to_datetime(df['Date'])
df = df.set_index('Date')

In [37]:
df.head()

,Sweden,Norway,Denmark,Finland
Date,,,,
2020-01-22,"{'date': '2020-1-22', 'confirmed': 0, 'deaths'...","{'date': '2020-1-22', 'confirmed': 0, 'deaths'...","{'date': '2020-1-22', 'confirmed': 0, 'deaths'...","{'date': '2020-1-22', 'confirmed': 0, 'deaths'..."
2020-01-23,"{'date': '2020-1-23', 'confirmed': 0, 'deaths'...","{'date': '2020-1-23', 'confirmed': 0, 'deaths'...","{'date': '2020-1-23', 'confirmed': 0, 'deaths'...","{'date': '2020-1-23', 'confirmed': 0, 'deaths'..."
2020-01-24,"{'date': '2020-1-24', 'confirmed': 0, 'deaths'...","{'date': '2020-1-24', 'confirmed': 0, 'deaths'...","{'date': '2020-1-24', 'confirmed': 0, 'deaths'...","{'date': '2020-1-24', 'confirmed': 0, 'deaths'..."
2020-01-25,"{'date': '2020-1-25', 'confirmed': 0, 'deaths'...","{'date': '2020-1-25', 'confirmed': 0, 'deaths'...","{'date': '2020-1-25', 'confirmed': 0, 'deaths'...","{'date': '2020-1-25', 'confirmed': 0, 'deaths'..."
2020-01-26,"{'date': '2020-1-26', 'confirmed': 0, 'deaths'...","{'date': '2020-1-26', 'confirmed': 0, 'deaths'...","{'date': '2020-1-26', 'confirmed': 0, 'deaths'...","{'date': '2020-1-26', 'confirmed': 0, 'deaths'..."


In [38]:
df_deaths = pd.DataFrame(index=df.index)
for col in df.columns:
    df_deaths[col] = [c.get('deaths') for c in df[col]]

In [39]:
df_deaths = df_deaths['2020-03-10':]

In [40]:
# population data from Wikipedia

df_pop = pd.read_html('https://en.wikipedia.org/wiki/List_of_countries_by_population_(United_Nations)')[3]
df_pop = df_pop[['Country or area', 'Population(1 July 2019)']]

df_pop['Country or area'] = df_pop['Country or area'].str.replace('\[.*\]','')
df_pop = df_pop.pivot_table(columns='Country or area',values='Population(1 July 2019)')[df.columns]
df_pop = df_pop / 1000000

In [41]:
df_pop['Sweden']

Population(1 July 2019)    10.036379
Name: Sweden, dtype: float64

In [42]:
df_deaths_per_mn = pd.DataFrame(index=df_deaths.index)
for col in df_deaths.columns:
    df_deaths_per_mn[col] = df_deaths[col] / df_pop[col].values

In [43]:
df_deaths_per_mn

,Sweden,Norway,Denmark,Finland
Date,,,,
2020-03-10,0.000000,0.000000,0.000000,0.000000
2020-03-11,0.099638,0.000000,0.000000,0.000000
2020-03-12,0.099638,0.000000,0.000000,0.000000
2020-03-13,0.099638,0.000000,0.000000,0.000000
2020-03-14,0.199275,0.557739,0.173254,0.000000
2020-03-15,0.298913,0.557739,0.346508,0.000000
2020-03-16,0.597825,0.557739,0.519762,0.000000
2020-03-17,0.697463,0.557739,0.693016,0.000000
2020-03-18,0.996375,1.115479,0.693016,0.000000


# Trend from first death

In [44]:
df_deaths_1 = df_deaths[df_deaths != 0]

In [45]:
df_deaths_1 = df_deaths_1.apply(lambda x: pd.Series(x.dropna().values))

In [46]:
df_deaths_1.head()

,Sweden,Norway,Denmark,Finland
0,1.0,3.0,1.0,1.0
1,1.0,3.0,2.0,1.0
2,1.0,3.0,3.0,1.0
3,2.0,3.0,4.0,1.0
4,3.0,6.0,4.0,3.0


In [47]:
df_deaths_per_mn_1 = pd.DataFrame(index=df_deaths_1.index)
for col in df_deaths.columns:
    df_deaths_per_mn_1[col] = df_deaths_1[col] / df_pop[col].values

# Plots

## Deaths over time

In [50]:
import plotly.graph_objects as go
import pandas as pd
import plotly


fig = go.Figure()

for col in df_deaths:
    fig.add_trace(go.Scatter(x=df_deaths.index, y=df_deaths[col], name=col))

fig.update_layout(title_text='COVID19 Nordic Deaths, starting March 10 2020',hovermode = 'x',
                  xaxis_rangeslider_visible=True)
fig.show()
plotly.offline.plot(fig, filename='deaths.html')

'deaths.html'

In [51]:
fig = go.Figure()

for col in df_deaths_per_mn:
    fig.add_trace(go.Scatter(x=df_deaths_per_mn.index, y=df_deaths_per_mn[col], name=col))

fig.update_layout(title_text='COVID19 Nordic Deaths per Mn inhabitants, starting March 10 2020',hovermode = 'x',
                  xaxis_rangeslider_visible=True)
fig.show()
plotly.offline.plot(fig, filename='deaths_mn.html')

'deaths_mn.html'

In [54]:
fig = go.Figure()

for col in df_deaths_1:
    fig.add_trace(go.Scatter(x=df_deaths_1.index, y=df_deaths_1[col], name=col))

fig.update_layout(title_text='COVID19 Nordic Deaths, increase per day since first death',hovermode = 'x',
                  xaxis_rangeslider_visible=True)
fig.show()
plotly.offline.plot(fig, filename='deaths_1.html')

'deaths_1.html'

In [55]:
fig = go.Figure()

for col in df_deaths_per_mn_1:
    fig.add_trace(go.Scatter(x=df_deaths_per_mn_1.index, y=df_deaths_per_mn_1[col], name=col))

fig.update_layout(title_text='COVID19 Nordic Deaths per Mn inhabitants, increase per day since first death',
                  xaxis_rangeslider_visible=True)
fig.show()
plotly.offline.plot(fig, filename='deaths_mn_1.html')

'deaths_mn_1.html'

In [ ]:
from datawrapper import Datawrapper
dw = Datawrapper(access_token = "ead65363820f97b7bf23049d4beecb4755f08b3b130efa6eb385f8931c303f67")

In [ ]:
dw.account_info()

In [ ]:
# Start from the date of the first death


df_deaths['2020-03-10':].plot(figsize=(10,10),grid=True)



In [ ]:
df_deaths_per_mn['2020-03-10':].plot()